In [ ]:

!pip install -q transformers datasets torch sentencepiece

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType

"""
Model adı ve tokenizer'ı yükle, cihazı belirle
"""
MODEL_NAME = "ytu-ce-cosmos/turkish-gpt2-large-750m-instruct-v0.1" #ytu-ce-cosmos/Turkish-Gemma-9b-v0.1
# bu modeller teacher olarak hazırladığım davranış verileriyle eğittim

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32
).to(device)

print("Model & tokenizer OK")

"""
Dataset'i yükle ve örnek veriyi göster
"""
dataset = load_dataset("bylang/behavior_data")

print(dataset)
print(dataset["train"][0])

"""
Tokenize fonksiyonu: Soru ve cevabı birleştirip tokenize et
"""
def tokenize_fn(example):
    text = example["question"] + "\n" + example["answer"]
    tokens = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

"""
Dataset'i tokenize et ve gereksiz sütunları kaldır
"""
tokenized_ds = dataset.map(
    tokenize_fn,
    remove_columns=dataset["train"].column_names
)

"""
LoRA konfigürasyonu ayarlama
"""
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["c_attn", "c_proj"]
)

"""
Model'e LoRA uygulayıp eğitilebilir parametreleri göster
"""
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

"""
Data collator oluştur
"""
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

"""
Çıktı dizini ve eğitim argümanlarını ayarla
"""
OUTPUT_DIR = "/content/drive/MyDrive/turkish_gpt2_peft"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to="none",
    optim="adamw_torch"
)

"""
Trainer'ı oluşturma
"""
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

"""
Modeli eğitme işlemi
"""
trainer.train()

"""
Eğitilmiş modeli Hugging Face Hub'a yükleme
"""
trainer.push_to_hub("--",token="--")# bylang/teacher1_cosmos_gpt2, bylang/teacher2_cosmos_gemma bu modelleri hf'ye pushladım

In [ ]:

import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer



"""
finetune ettiğim ve etmediğim teacher modellerine rastgele oluşturduğum promptları sorup cevap aldım
"""
HF_REPO = "--" #bylang/teacher2_cosmos_gemma , bylang/teacher1_cosmos_gpt2 , ytu-ce-cosmos/turkish-gpt2-large-750m-instruct-v0.1, ytu-ce-cosmos/Turkish-Gemma-9b-v0.1
teacher_model = AutoModelForCausalLM.from_pretrained(HF_REPO)
teacher_tokenizer = AutoTokenizer.from_pretrained(HF_REPO)
teacher_model.eval()

"""
Cihazı belirle: GPU varsa kullan, yoksa CPU
"""
# GPU varsa kullan, yoksa CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
teacher_model.to(device)



"""
CSV dosyasından prompt'ları okuma ve listeye çevirme işlemi
"""
csv_path = "/kaggle/input/dataset/random_student_prompts.csv"
df_prompts = pd.read_csv(csv_path)
prompts = df_prompts["input"].tolist()



"""
Her prompt için teacher model ile çıktı üretme
"""
teacher_outputs = []

for i, prompt in enumerate(prompts, 1):
    inputs = teacher_tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output_ids = teacher_model.generate(
            **inputs,
            max_length=50,
            do_sample=True,
            top_p=0.9,
            temperature=0.8
        )
    output_text = teacher_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    teacher_outputs.append(output_text)
    
    if i % 20 == 0:
        print(f"{i}/{len(prompts)} prompt işlendi...")



"""
Student dataset'ini oluşturma ve CSV olarak kaydetme işlemi
"""
df_student = pd.DataFrame({"input": prompts, "target": teacher_outputs})
df_student.to_csv("/kaggle/working/student_dataset.csv", index=False)

print("Student dataset oluşturuldu ve 'student_dataset.csv' olarak kaydedildi.")